### Вспомогательная функция
$
f_t(x,\mu) = tf(x, \mu) + F(x, \mu) = \\
t(\frac{1}{2}||Ax-b||_2^2+\lambda \langle 1, u\rangle) + \sum_{i=1}^n (- \ln(-(x_i - \mu_i)) - \ln(-(-x_i-\mu_i))) = \\
t(\frac{1}{2}||Ax-b||_2^2+\lambda \langle 1, u\rangle) + \sum_{i=1}^n (- \ln (\mu_i - x_i) - \ln(x_i+\mu_i))
$

### Направление
$
d_k = (d_k^x, d_k^\mu)\\
d_k^x = -[\nabla^2f_t(x, \mu)_x]^{-1}\nabla f_t(x, \mu)_x\\
d_k^\mu = -[\nabla^2f_t(x, \mu)_\mu]^{-1}\nabla f_t(x, \mu)_\mu\\
$

#### Для х:
$\\
d^xf_t(x,\mu) = 
t \langle Ax - b, Adx\rangle + \sum_{i=1}^n (\frac{1}{\mu_i - x_i} - \frac{1}{\mu_i + x_i})d_{x_i}
=\\
t \langle A^T(Ax - b), dx\rangle + \sum_{i=1}^n (\frac{1}{\mu_i - x_i} - \frac{1}{\mu_i + x_i})d_{x_i}
\\
\nabla_x f_t(x, \mu)_i = (tA^T(Ax - b))_i + \frac{1}{\mu_i - x_i} - \frac{1}{\mu_i + x_i}
\\
d [f_t(x, \mu)][dx, dx] = 
t \langle A^TAdx, dx\rangle + \sum_{i=1}^n (\frac{1}{(\mu_i - x_i)^2} + \frac{1}{(\mu_i + x_i)^2})d_{x_i}
\\
\nabla^2_x f_t(x, \mu)_{ii} = (tA^TA)_{ii} + \frac{1}{(\mu_i - x_i)^2} + \frac{1}{(\mu_i + x_i)^2}\\
\nabla^2_x f_t(x, \mu)_{ij, i\ne j} = 
(tA^TA)_{ij, i\ne j}$

#### Для $\mu:\\$
$
d^\mu f_t(x,\mu) = 
t\lambda \langle 1, d\mu\rangle + \sum_{i=1}^n (- \frac{1}{\mu_i - x_i} - \frac{1}{x_i+\mu_i})d_{\mu_i}
\\
\nabla_\mu f_t(x, \mu)_i = t\lambda - \frac{1}{\mu_i - x_i} - \frac{1}{x_i+\mu_i}
\\
d^\mu f_t(x,\mu)[d\mu d\mu] = d(\sum_{i=1}^n (- \frac{1}{\mu_i - x_i} - \frac{1}{x_i+\mu_i}))=
\sum_{i=1}^n (\frac{1}{(\mu_i - x_i)^2} + \frac{1}{(x_i+\mu_i)^2})d_{\mu_i} d_{\mu_i}
\\
\nabla^2_\mu f_t(x, \mu)_{ii} = \frac{1}{(\mu_i - x_i)^2} + \frac{1}{(x_i+\mu_i)^2}
\\
\nabla^2_\mu f_t(x, \mu)_{ij, i\ne j} = 0$

#### Смешанная часть гессиана:
$
\nabla^2_\mu f_t(x, \mu)_{i_x, i_\mu} =
d/d^{\mu_i}(\frac{1}{\mu_i - x_i} - \frac{1}{\mu_i + x_i}) = 
\frac{1}{(\mu_i + x_i)^2} - \frac{1}{(\mu_i - x_i)^2}
\\
\nabla^2_\mu f_t(x, \mu)_{i_\mu, i_x} =
d/d^{x_i}(- \frac{1}{\mu_i - x_i} - \frac{1}{x_i+\mu_i}) = 
\frac{1}{(x_i+\mu_i)^2} - \frac{1}{(\mu_i - x_i)^2} = \nabla^2_\mu f_t(x, \mu)_{i_x, i_\mu}
$

### Максимальное $\alpha$

$g_i(x, \mu) = \langle q, (x, \mu) \rangle$, где $(x, \mu)$ - вектор состоящий из х и $\mu$, а $q = (..., \pm 1,..., -1, ...)$ это вектор где на $i_x (=i)$ месте стоит коэфицент для $x_i$, на $i_\mu (=i + n)$ стоит коэфицент для $\mu_i$, а в остальных местах 0.

Тогда $\alpha_{\max} = \min\limits_{i\in I} \frac{\langle q_i, (x, \mu) \rangle}{\langle q_i, d \rangle} = \min\limits_{i\in I} \frac{sgn(q_{ij})x_j - \mu_j}{sgn(q_{ij})d_j-d_{j+n}}$, где $I$ - множество индексов, где знаменатель больше 0.

### Начальная точка

В данной задаче нам нужно выбрать подходящую под условия начальную точку: $x = 0_n, \mu = 1_n$

### Способ решения

Предлагаю взять для способа решения `LBFGS`, чтобы наши вычисления поместились в RAM и не было потребности вычислять гессиан.